# Project 2

## Story

This project will use the dataset of video game sales from 2016 to determine whether or not a game was a hit or not a hit depending on how many copies they sold that week. The purpose of this is to see any patterns among games that are hits

In [ ]:
%pip install numpy pandas matplotlib seaborn 

In [2]:
import sklearn
import numpy as py
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
game_sales_df = pd.read_csv('Video_Games_Sales_as_at_22_Dec_2016.csv')
print(game_sales_df)

                                Name Platform  Year_of_Release         Genre  \
0                         Wii Sports      Wii           2006.0        Sports   
1                  Super Mario Bros.      NES           1985.0      Platform   
2                     Mario Kart Wii      Wii           2008.0        Racing   
3                  Wii Sports Resort      Wii           2009.0        Sports   
4           Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing   
...                              ...      ...              ...           ...   
16714  Samurai Warriors: Sanada Maru      PS3           2016.0        Action   
16715               LMA Manager 2007     X360           2006.0        Sports   
16716        Haitaka no Psychedelica      PSV           2016.0     Adventure   
16717               Spirits & Spells      GBA           2003.0      Platform   
16718            Winning Post 8 2016      PSV           2016.0    Simulation   

          Publisher  NA_Sales  EU_Sales

In [4]:
game_sales_df.isnull().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Developer          6623
Rating             6769
dtype: int64

After checking if the table has null values, I need to decide whether or not it's worth removing those rows and columns.

For my classification part, I'll be using a decision tree because of how easy it is for others to interpret it and can handle both numerical and categorical data. The most important columns I'll need to train my tree is the game's name, its global sales, and the target variable which I will generate. To determine whether or not a game is a "hit" or not, I will use the threshold of 1 million in global sales because this is the common industry standard they use to determine whether or not a game is a success. I will also have to engineer the categorical features into features where they are binary so the machine learning model can understand it.